<a href="https://colab.research.google.com/github/11Alejandro/Trabajo-final---Parqueadero/blob/main/src/Gesti%C3%B3n_de_parqueadero_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================== LIBRERIAS Y RUTA =============================
from datetime import datetime
import os
import pytz
import time

ARCHIVO_DATOS = "parqueadero.txt"

usuarios = {}
vehiculos_en_el_parqueadero = {}
historial_retiros = []
usuarios_admin = {"admin": "admin123", "profesor": "profe456"}
tiempos_actuales = []

# ========================== FUNCIONES DE GUARDADO/CARGA =======================

def guardar():
    with open(ARCHIVO_DATOS, "w") as f:
        f.write("# USUARIOS\n")
        for doc, data in usuarios.items():
            f.write(f"{doc},{data['nombre']},{data['apellido']},{data['placa']}\n")

        f.write("\n# VEHICULOS_PARQUEADOS\n")
        for doc, data in vehiculos_en_el_parqueadero.items():
            f.write(f"{doc},{data['hora_ingreso'].strftime('%Y-%m-%d %H:%M:%S')},{data['placa']}\n")

        f.write("\n# HISTORIAL_RETIROS\n")
        for item in historial_retiros:
            f.write(f"{item['documento']},{item['tiempo_parqueado_minutos']},{item['total_pagado']}\n")


def cargar_todo_desde_un_archivo():
    global usuarios, vehiculos_en_el_parqueadero, historial_retiros
    usuarios = {}
    vehiculos_en_el_parqueadero = {}
    historial_retiros = []

    if not os.path.exists(ARCHIVO_DATOS):
        return

    with open(ARCHIVO_DATOS, "r") as f:
        seccion = None
        for linea in f:
            linea = linea.strip()
            if not linea or linea.startswith("#"):
                if "USUARIOS" in linea:
                    seccion = "usuarios"
                elif "VEHICULOS_PARQUEADOS" in linea:
                    seccion = "parqueados"
                elif "HISTORIAL_RETIROS" in linea:
                    seccion = "retiros"
                continue

            if seccion == "usuarios":
                doc, nombre, apellido, placa = linea.split(",")
                usuarios[doc] = {
                    "nombre": nombre,
                    "apellido": apellido,
                    "documento_identidad": doc,
                    "placa": placa
                }

            elif seccion == "parqueados":
                doc, hora_str, placa = linea.split(",")
                hora_ingreso = datetime.strptime(hora_str, "%Y-%m-%d %H:%M:%S")
                vehiculos_en_el_parqueadero[doc] = {
                    "hora_ingreso": hora_ingreso,
                    "placa": placa
                }

            elif seccion == "retiros":
                doc, minutos, total = linea.split(",")
                historial_retiros.append({
                    "documento": doc,
                    "tiempo_parqueado_minutos": int(minutos),
                    "total_pagado": int(total)
                })

# ======================== FUNCIONES DE VALIDACIÓN =============================

# Validación de nombres
def validar_nombre(nombre):
    if len(nombre) < 3:
        print("\033[3;31mEl nombre debe tener al menos 3 letras. Por favor, inténtelo otra vez.\033[0m")
        return False
    if not nombre.isalpha():
        print("\033[3;31mEl nombre solo debe contener letras sin espacios, números ni símbolos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación de apellidos
def validar_apellido(apellido):
    if len(apellido) < 3:
        print("\033[3;31mEl apellido debe tener al menos 3 letras. Por favor, inténtelo otra vez.\033[0m")
        return False
    if not apellido.isalpha():
        print("\033[3;31mEl apellido solo debe contener letras sin espacios, números ni símbolos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación del documento de identidad
def validar_documento(documento):
    if not documento.isdigit():
        print("\033[3;31mEl documento solo debe contener números. Por favor, inténtelo otra vez.\033[0m")
        return False
    if len(documento) < 3 or len(documento) > 15:
        print("\033[3;31mEl documento debe tener entre 3 y 15 dígitos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación de la placa
def validar_placa(placa):
    placa = placa.upper()
    if len(placa) != 6:
        print("\033[3;31mLa placa debe tener exactamente 6 caracteres. Por favor, inténtelo otra vez.\033[0m")
        return False
    letras = placa[:3]
    numeros = placa[3:]
    if not letras.isalpha() or not numeros.isdigit():
        print("\033[3;31mLa placa debe tener 3 letras seguidas de 3 números. Ej: ABC123. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# =============== FUNCIÓN Y PROCEDIMIENTO PARA REGISTRAR USUARIOS ==============

def registrar_nuevo_usuario():
    print("\n\033[1m------------------ REGISTRO DE NUEVO USUARIO --------------------\033[0m")

    # ------- Nombre -------
    while True:
        nombre = input("Ingrese su nombre: ")
        if validar_nombre(nombre):
            break

    #------- Apellido -------
    while True:
        apellido = input("Ingrese su apellido: ")
        if validar_apellido(apellido):
            break

    # ------- Documento con validación de duplicados -------
    while True:
        documento = input("Ingrese su número de documento: ")
        if not validar_documento(documento):
            continue

        if documento in usuarios:
            print(f"\033[3;31mYa existe un usuario registrado con el documento {documento}.\033[0m")
            decision = input("¿Desea intentar con otro documento? (si / no): ").lower()
            if decision == "no":
                print("\033[3;33mRegresando al menú principal...\033[0m")
                return
        else:
            break

    # ------- Placa -------
    while True:
        placa = input("Ingrese la placa del vehículo (formato ABC123): ")
        if validar_placa(placa):
            placa = placa.upper()
            break

    # -------Resumen y opción de corrección -------
    while True:
        print("\n\033[1;32m-------------- Resumen de registro -----------------\033[0m")
        print(f"▪ Documento: {documento}")
        print(f"▪ Nombre: {nombre}")
        print(f"▪ Apellido: {apellido}")
        print(f"▪ Placa del vehículo: {placa}")
        print("\033[1;32m----------------------------------------------------\033[0m")

        respuesta = input("\n¿Desea modificar algún dato? (si / no): ").lower()
        if respuesta == "no":
            break

        campo = input("¿Qué desea modificar? (nombre / apellido / documento / placa): ").lower()
        if campo == "nombre":
            while True:
                nuevo = input("Nuevo nombre: ")
                if validar_nombre(nuevo):
                    nombre = nuevo
                    break
        elif campo == "apellido":
            while True:
                nuevo = input("Nuevo apellido: ")
                if validar_apellido(nuevo):
                    apellido = nuevo
                    break
        elif campo == "documento":
            while True:
                nuevo = input("Nuevo número de documento: ")
                if validar_documento(nuevo) and nuevo not in usuarios:
                    documento = nuevo
                    break
                elif nuevo in usuarios:
                    print("\033[3;31mEste documento ya está registrado. Ingrese otro.\033[0m")
        elif campo == "placa":
            while True:
                nuevo = input("Nueva placa (formato ABC123): ")
                if validar_placa(nuevo):
                    placa = nuevo.upper()
                    break
        else:
            print("\033[3;31mOpción inválida. Por favor escriba solo: nombre, apellido, documento o placa.\033[0m")

    # ------- Guardar datos y finalizar registro -------
    usuarios[documento] = {
        "nombre": nombre,
        "apellido": apellido,
        "documento_identidad": documento,
        "placa": placa
    }

    print("\033[1;32m------ El usuario ha sido registrado exitosamente ------\033[0m")

    guardar()

    opcion = input("\n¿Desea registrar otro vehículo? (si / no): ").lower()
    if opcion == "si":
        registrar_nuevo_usuario()



# ================ FUNCIÓN Y PROCEDIMIENTO PARA INGRESAR VEHICULO ==============

def ingresar_vehiculo():
    while True:
        print("\n\033[1m=============== INGRESO DE VEHÍCULO ===============\033[0m")

        documento = input("Ingrese el número de documento del usuario: ")

        if documento not in usuarios:
            print("\033[3;31mEste documento no está registrado.\033[0m")
            opcion = input("¿Desea registrar el usuario? (si/no):").lower()
            if opcion == "si":
                registrar_nuevo_usuario()
            else:
                print("\033[3;31mNo se puede continuar sin un usuario registrado.\033[0m")
                return

        if documento in vehiculos_en_el_parqueadero:
            print("\033[3;31mEste usuario ya tiene un vehículo dentro del parqueadero.\033[0m")
            return

        if len(vehiculos_en_el_parqueadero) >= 64:
            print("\033[3;31mEn este momento no hay celdas disponibles.\033[0m")
            return

        # Obtener fecha y hora actual con zona horaria de Bogotá
        zona_colombia = pytz.timezone("America/Bogota")
        fecha_hora_actual = datetime.now(zona_colombia)


        vehiculos_en_el_parqueadero[documento] = {"hora_ingreso": fecha_hora_actual}

        nombre = usuarios[documento]["nombre"]
        apellido = usuarios[documento]["apellido"]
        placa = usuarios[documento]["placa"]
        hora_ingreso = fecha_hora_actual.strftime("%Y-%m-%d %H:%M:%S")

        print(f"\n\033[1;32m-------------- Resumen de ingreso ----------------\033[0m")
        print(f"▪ Documento:{documento}")
        print(f"▪ Nombre:{nombre}")
        print(f"▪ Apellido:\033[0m {apellido}")
        print(f"▪ Placa del vehículo:{placa}")
        print(f"▪ Hora de ingreso:{hora_ingreso}")
        print(f"\033[1;32m---------------------------------------------------\033[0m")


       # Guardar después del ingreso
        guardar()


        opcion = input("\n¿Desea ingresar otro vehículo? (si / no): ").lower()
        if opcion != "si":
         break

# =============== FUNCIÓN Y PROCEDIMIENTO PARA RETIRAR VEHICULO ================

def retirar_vehiculo():
    print("\n\033[1m=============== RETIRO DE VEHÍCULO ===============\033[0m")

    usuario = input("Ingrese el número de documento del usuario: ")

    if usuario not in usuarios:
        print("No hay ningún usuario registrado con este documento.")
        opcion = input("¿Desea registrarse? (si/no): ").lower()
        if opcion == "si":
            registrar_nuevo_usuario()
        return

    if usuario not in vehiculos_en_el_parqueadero:
        print("No hay ningún vehículo registrado actualmente en el parqueadero con este documento.")
        return

    zona_colombia = pytz.timezone("America/Bogota")
    hora_ingreso = vehiculos_en_el_parqueadero[usuario]["hora_ingreso"]
    hora_salida = datetime.now(zona_colombia)

    # Asegurar que la hora de ingreso tenga zona horaria
    if hora_ingreso.tzinfo is None:
        hora_ingreso = zona_colombia.localize(hora_ingreso)

    tiempo_parqueado = int((hora_salida - hora_ingreso).total_seconds() // 60)

    valor_hora = 7000
    valor_cuarto = 1500

    horas = tiempo_parqueado // 60
    minutos_restantes = tiempo_parqueado % 60

    if minutos_restantes == 0:
        cuartos = 0
    elif minutos_restantes <= 15:
        cuartos = 1
    elif minutos_restantes <= 30:
        cuartos = 2
    elif minutos_restantes <= 45:
        cuartos = 3
    else:
        cuartos = 4

    total_horas = horas * valor_hora
    total_cuartos = cuartos * valor_cuarto
    total_a_pagar = total_horas + total_cuartos

    if total_a_pagar < valor_hora:
        total_a_pagar = valor_hora
        print("\n\033[1mSe aplicó el pago mínimo de 7000 pesos.\033[0m")

    print("\n\033[1;32m-------------- Resumen del retiro ----------------\033[0m")
    print("Hora de ingreso:", hora_ingreso.strftime('%Y-%m-%d %H:%M:%S'))
    print("Hora de salida:", hora_salida.strftime('%Y-%m-%d %H:%M:%S'))
    print("Horas completas:", horas)
    print("Minutos restantes:", minutos_restantes)
    print("Cuartos de hora a cobrar:", cuartos)
    print("Cobro por horas:", total_horas)
    print("Cobro por cuartos:", total_cuartos)
    print("Total a pagar por el parqueo:", total_a_pagar, "pesos")
    print(f"\033[1;32m--------------------------------------------------\033[0m")


    # Eliminar el vehículo del parqueadero
    del vehiculos_en_el_parqueadero[usuario]

    # Guardar en el historial de retiros
    historial_retiros.append({
        "documento": usuario,
        "tiempo_parqueado_minutos": tiempo_parqueado,
        "total_pagado": total_a_pagar
    })

    guardar()


# ==================== ACCESO AL MÓDULO DE ADMINISTRACIÓN ======================

# ---------------------- Módulo de administración ------------------------------

def iniciar_sesion_admin():
    print("\n\033[1m=============== MÓDULO DE ADMINISTRACIÓN/REPORTES ===============\033[0m")


    while True:
        usuario = input("Ingrese su usuario: ")
        if usuario not in usuarios_admin:
            print("\033[3;31mUsuario incorrecto.\033[0m")
            continue

        contrasena = input("Ingrese su contraseña: ")
        if usuarios_admin[usuario] == contrasena:
            return True
        else:
            print("\033[3;31mContraseña incorrecta.\033[0m")
            continue

# --------------------- Reporte de vehículos registrados -----------------------

def generar_reporte_total_vehiculos_registrados():
    print("\n\033[1;32m-------------- Reporte: Total de vehículos registrados --------------\033[0m")
    total = len(usuarios)
    print(f"▪ Total de usuarios con vehículo registrado: {total}")
    print("\033[1;32m---------------------------------------------------------------------\033[0m")

# --------------------- Reporte de vehículos retirados -------------------------

def generar_reporte_total_vehiculos_retirados():
    print("\n\033[1;32m-------------- Reporte: Total de vehículos retirados ---------------\033[0m")
    total = len(historial_retiros)
    print(f"▪ Total de vehículos que han sido retirados: {total}")
    print("\033[1;32m--------------------------------------------------------------------\033[0m")

# --------------------- Reporte de vehículos sin retirar -----------------------

def generar_reporte_total_vehiculos_sin_retirar():
    print("\n\033[1;32m----------- Reporte: Vehículos actualmente en el parqueadero -----------\033[0m")
    total = len(vehiculos_en_el_parqueadero)
    print(f"▪ Total de vehículos en el parqueadero: {total}")
    print("\033[1;32m----------------------------------------------------------------------\033[0m")

# ------------------- Reporte de pago de vehículos retirados -------------------

def generar_reporte_total_pago_vehiculos_retirados():
    print("\n\033[1;32m------------ Reporte: Total recaudado por retiros ------------\033[0m")
    total_pagado = sum(item['total_pagado'] for item in historial_retiros)
    print(f"▪ Monto total recaudado: ${total_pagado:,.2f} pesos")
    print("\033[1;32m--------------------------------------------------------------\033[0m")

# --------------------- Reporte de promedio de estancia ------------------------

def generar_reporte_tiempo_promedio_estancia():
    print("\n\033[1;32m----- Reporte: Tiempo promedio de estancia por vehículo -----\033[0m")
    if not historial_retiros:
        print("\033[3;33mNo hay vehículos retirados para calcular el promedio de estancia.\033[0m")
        return

    total_minutos = sum(item['tiempo_parqueado_minutos'] for item in historial_retiros)
    promedio_minutos = total_minutos / len(historial_retiros)
    promedio_horas = promedio_minutos / 60
    print(f"▪ Promedio: {promedio_minutos:.2f} minutos ({promedio_horas:.2f} horas)")
    print("\033[1;32m--------------------------------------------------------------\033[0m")

# --------------------- Reporte de usuarios registrados ------------------------

def generar_reporte_lista_usuarios():
    print("\n\033[1;32m------------- Reporte: Lista de usuarios registrados -------------\033[0m")
    if not usuarios:
        print("\033[3;33mNo hay usuarios registrados en el sistema.\033[0m")
        return
    for doc, data in usuarios.items():
        print(f"▪ Documento: {doc}, Nombre: {data['nombre']} {data['apellido']}, Placa: {data['placa']}")
    print("\033[1;32m------------------------------------------------------------------\033[0m")

# --------------- Reporte de tiempo máximo y mínimo de parqueo -----------------

def generar_reporte_vehiculo_tiempo_maximo_minimo_parqueo():
    print("\n\033[1;32m----- Reporte: Tiempo de parqueo máximo y mínimo (actuales) -----\033[0m")
    if not vehiculos_en_el_parqueadero:
        print("\033[3;33mNo hay vehículos actualmente en el parqueadero.\033[0m")
        return

    zona_colombia = pytz.timezone("America/Bogota")
    ahora = datetime.now(zona_colombia)

    tiempos_actuales.clear()
    for doc, data in vehiculos_en_el_parqueadero.items():
        tiempo_delta = ahora - data["hora_ingreso"]
        minutos_actuales = int(tiempo_delta.total_seconds() / 60)
        placa = data.get("placa", "N/A")
        tiempos_actuales.append({"documento": doc, "placa": placa, "minutos": minutos_actuales})

    if not tiempos_actuales:
        print("\033[3;33mNo hay tiempos disponibles para comparar.\033[0m")
        return

    vehiculo_max_tiempo = max(tiempos_actuales, key=lambda x: x['minutos'])
    vehiculo_min_tiempo = min(tiempos_actuales, key=lambda x: x['minutos'])

    print(f"▪ Vehículo con más tiempo:")
    print(f"  ▪ Placa: {vehiculo_max_tiempo['placa']}, Documento: {vehiculo_max_tiempo['documento']}")
    print(f"  ▪ Tiempo: {vehiculo_max_tiempo['minutos']} minutos")
    print(f"▪ Vehículo con menos tiempo:")
    print(f"  ▪ Placa: {vehiculo_min_tiempo['placa']}, Documento: {vehiculo_min_tiempo['documento']}")
    print(f"  ▪ Tiempo: {vehiculo_min_tiempo['minutos']} minutos")
    print("\033[1;32m--------------------------------------------------------------\033[0m")

# ----------------------- Menú del módulo de administración --------------------

def menu_administracion():
    while True:
        print(r"""



        """ + "\033[1m" + "  ¡BIENVENIDO AL MENÚ DEL MÓDULO DE ADMINISTRACIÓN!" + "\033[0m")
        print("\n\033[1m==================== MENÚ DE ADMINISTRACIÓN ====================\033[0m")
        print("1. Total de vehículos registrados (usuarios)")
        print("2. Total de vehículos retirados (historial)")
        print("3. Total de vehículos sin retirar (actualmente en parqueadero)")
        print("4. Total pago de vehículos retirados")
        print("5. Tiempo promedio de estancia por vehículo en el parqueadero")
        print("6. Lista de usuarios registrados")
        print("7. Vehículo con tiempo de parqueo máximo y mínimo (actuales)")
        print("8. Salir del menú de administración")
        opcion = input("Seleccione una opción (1-8): ")
        print("\033[1m==================================================================\033[0m")


        if opcion == "1":
            generar_reporte_total_vehiculos_registrados()
        elif opcion == "2":
            generar_reporte_total_vehiculos_retirados()
        elif opcion == "3":
            generar_reporte_total_vehiculos_sin_retirar()
        elif opcion == "4":
            generar_reporte_total_pago_vehiculos_retirados()
        elif opcion == "5":
            generar_reporte_tiempo_promedio_estancia()
        elif opcion == "6":
            generar_reporte_lista_usuarios()
        elif opcion == "7":
            generar_reporte_vehiculo_tiempo_maximo_minimo_parqueo()
        elif opcion == "8":
            print("\033[1;32mSaliste del módulo de administración. ¡Hasta pronto!\033[0m")
            break
        else:
            print("\033[3;31mOpción inválida. Intente de nuevo.\033[0m")

print("\033[1;32m.\033[0m")

# ============================= MENÚ PRINCIPAL =================================

def menu_parqueadero():
    while True:
        print(r"""
                            ______
                           /|_||_\`.__
                          (   _    _ _\
                          =`-(_)--(_)-'

         """ + "\033[1m" + "       ¡BIENVENIDO AL PARQUEADER0 AUTOPARK!" + "\033[0m")

        print("\033[1m======================== MENÚ PRINCIPAL ========================\033[0m")
        print("1. Registrar usuario")
        print("2. Ingreso del vehículo")
        print("3. Retiro del vehículo")
        print("4. Módulo de administración/reportes")
        print("5. Salir")
        opcion = input("Selecciona una opción (1-5): ")
        print("\033[1m==================================================================\033[0m")

        if opcion == "1":
            registrar_nuevo_usuario()
            guardar()
        elif opcion == "2":
            ingresar_vehiculo()
            guardar()
        elif opcion == "3":
            retirar_vehiculo()
            guardar()
        elif opcion == "4":
            if iniciar_sesion_admin():
                menu_administracion()
        elif opcion == "5":
            print("\033[1;34mSaliste del menú principal. ¡Hasta pronto!\033[0m")
            guardar()
            break
        else:
            print("Opción inválida. Intente de nuevo.")

# ========================== INICIO DEL PROGRAMA ===============================

cargar_todo_desde_un_archivo()
guardar()
menu_parqueadero()
